In [1]:
import torch.nn as nn
import torch.optim as optim
import torchvision
import torch

In [2]:
import gc 
gc.collect()
torch.cuda.empty_cache()

In [3]:
a = torch.rand(size=(32,3,32,32))

In [4]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR100(root='C:\\Users\\get2b\\Desktop\\Arav\\AI ML\\DATA SETS\\pytorch Data',train=True,download=False,transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=4,shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR100(root='C:\\Users\\get2b\\Desktop\\Arav\\AI ML\\DATA SETS\\pytorch Data', train=False,download=False, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=1000,shuffle=False, num_workers=2)

In [5]:
alex = nn.Sequential(nn.Conv2d(3,64,3,1,1),
                                  nn.Conv2d(64,64,3,1,1),
                                  nn.MaxPool2d(3,2,1),
                                  nn.Conv2d(64,128,3,1,1),
                                  nn.Conv2d(128,128,3,1,1),
                                  nn.MaxPool2d(3,2,1),
                                  nn.Conv2d(128,256,3,1,1),
                                  nn.Conv2d(256,256,3,1,1),
                                  nn.Conv2d(256,256,1,1),
                                  nn.MaxPool2d(3,2,1),
                                  nn.Conv2d(256,512,3,1,1),
                                  nn.Conv2d(512,512,3,1,1),
                                  nn.Conv2d(512,512,1,1),
                                  nn.MaxPool2d(3,2,1),
                                  nn.Conv2d(512,512,3,1,1),
                                  nn.Conv2d(512,512,3,1,1),
                                  nn.Conv2d(512,512,1,1),
                                  nn.Flatten())(a)

In [6]:
alex.shape

torch.Size([32, 2048])

In [7]:
class Alexnet(nn.Module):
    def __init__(self) :
        super().__init__()
        self.alex = nn.Sequential(nn.Conv2d(3,64,3,1,1),
                                  nn.BatchNorm2d(64,eps=0.001),nn.ReLU(),
                                  nn.Conv2d(64,64,3,1,1),
                                  nn.BatchNorm2d(64,eps=0.001),nn.ReLU(),
                                  nn.MaxPool2d(3,2,1),
                                  nn.Conv2d(64,128,3,1,1),
                                  nn.BatchNorm2d(128,eps=0.001),nn.ReLU(),
                                  nn.Conv2d(128,128,3,1,1),
                                  nn.BatchNorm2d(128,eps=0.001),nn.ReLU(),
                                  nn.MaxPool2d(3,2,1),
                                  nn.Conv2d(128,256,3,1,1),
                                  nn.BatchNorm2d(256,eps=0.001),nn.ReLU(),
                                  nn.Conv2d(256,256,3,1,1),
                                  nn.BatchNorm2d(256,eps=0.001),nn.ReLU(),
                                  nn.Conv2d(256,256,1,1),
                                  nn.BatchNorm2d(256,eps=0.001),nn.ReLU(),
                                  nn.MaxPool2d(3,2,1),
                                  nn.Conv2d(256,512,3,1,1),
                                  nn.BatchNorm2d(512,eps=0.001),nn.ReLU(),
                                  nn.Conv2d(512,512,3,1,1),
                                  nn.BatchNorm2d(512,eps=0.001),nn.ReLU(),
                                  nn.Conv2d(512,512,1,1),
                                  nn.BatchNorm2d(512,eps=0.001),nn.ReLU(),
                                  nn.MaxPool2d(3,2,1),
                                  nn.Conv2d(512,512,3,1,1),
                                  nn.BatchNorm2d(512,eps=0.001),nn.ReLU(),
                                  nn.Conv2d(512,512,3,1,1),
                                  nn.BatchNorm2d(512,eps=0.001),nn.ReLU(),
                                  nn.Conv2d(512,512,1,1),
                                  nn.BatchNorm2d(512,eps=0.001),nn.ReLU(),
                                  nn.Flatten(),
                                  nn.Linear(2048,4096),nn.ReLU(),
                                  nn.Dropout(p=0.5),
                                  nn.Linear(4096,4096),nn.ReLU(),
                                  nn.Dropout(p=0.5),
                                  nn.Linear(4096,1000),nn.ReLU())
    def forward(self,x):
        x = self.alex(x)
        return x

In [15]:
model = Alexnet().cuda()
try:
    model.load_state_dict(torch.load('C:\\Users\\get2b\\Desktop\\Arav\\AI ML\\Neural Networks\\Pytorch implementation\\All Models Saved\\AlexNet.pth'))
except:
    print("No saved model")

No saved model


In [16]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=1e-3,momentum=0.5)
lr_scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer=optimizer,
                                                 base_lr=1e-4,
                                                 max_lr=1e-3,
                                                 step_size_up=5,
                                                 step_size_down=5,
                                                 mode="exp_range",
                                                 cycle_momentum=False)

In [19]:
def test():
    model.eval()
    total = 0.0
    accuracy = 0.0
    with torch.no_grad():
        
        for i ,data in enumerate(test_loader,0):
            
            inputs, labels = data
            inputs, labels = inputs.cuda(),labels.cuda()
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total+=labels.size(0)
            accuracy+=(predicted == labels).sum().item()
    
    print("{}/{}".format(accuracy,total))
    accuracy = float(100 * accuracy / total)
    
    return(accuracy)



def train(epoch):
    for ep in range(1,epoch+1):
        print("Epoch : ",ep)
        running_loss=0
        for i, data in enumerate(train_loader,0):
            inputs,labels = data
            inputs,labels = inputs.cuda(),labels.cuda()
            optimizer.zero_grad(set_to_none=False)
            outputs = model(inputs)
            #outputs_googLeNet = torch.add(torch.add(outputs[0]*0.4, outputs[1]*0.3).cuda(),outputs[2]*0.3).cuda()
            loss = loss_function(outputs,labels)
            loss.backward()
            optimizer.step()
            running_loss+=loss.item()
            if (i+1)%1000 == 0 :
                print("[{}/{}]".format(i+1,12000))
                print("Loss of the training model is : %.4f"%(running_loss/1000))
                running_loss=0
        lr_scheduler.step()
        accuracy = test()
        print('For epoch', ep,'the test accuracy over the whole test set is %.2f %%' % (accuracy))

In [20]:
train(25)

Epoch :  1
[1000/50000]
Loss of the training model is : 4.6344
[2000/50000]
Loss of the training model is : 4.5660
[3000/50000]
Loss of the training model is : 4.4955
[4000/50000]
Loss of the training model is : 4.4634
[5000/50000]
Loss of the training model is : 4.4058
[6000/50000]
Loss of the training model is : 4.3366
[7000/50000]
Loss of the training model is : 4.2971
[8000/50000]
Loss of the training model is : 4.2246
[9000/50000]
Loss of the training model is : 4.1793
[10000/50000]
Loss of the training model is : 4.1102
[11000/50000]
Loss of the training model is : 4.0971
[12000/50000]
Loss of the training model is : 4.0313
907.0/10000.0
For epoch 1 the test accuracy over the whole test set is 9.07 %
Epoch :  2
[1000/50000]
Loss of the training model is : 3.8773
[2000/50000]
Loss of the training model is : 3.8083
[3000/50000]
Loss of the training model is : 3.7032
[4000/50000]
Loss of the training model is : 3.6474
[5000/50000]
Loss of the training model is : 3.5892
[6000/50000]


KeyboardInterrupt: 

In [24]:
torch.save(model.state_dict(),r'C:\Users\get2b\Desktop\Arav\Neural Networks\Pytorch implementation\All Models Saved\AlexNet.pth')